In [1]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model

In [2]:
model = load_model('Model/Model_LSTM.h5', compile=False)
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [3]:
actions = np.array(['Hello', 'Hungry', 'Sick', 'Sorry', 'Thank you', 'What', 'When', 'Where', 'Who', 'Why'])

In [15]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )
    
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

In [24]:
def play_video(name_video, model, actions):
  sequence = []
  with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    cap = cv2.VideoCapture("Test Video/New/{0}.mp4".format(name_video))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    vdo_writer = cv2.VideoWriter('Test Video/New/{0}.mp4'.format(name_video + "LSTM"), fourcc, 30.0, (800, 480))
    n = 0
    while True:
      n += 1
      ret, frame = cap.read()
      if ret:
        frame = cv2.resize(frame, (800, 480))
        image, results = mediapipe_detection(frame, holistic)
        draw_styled_landmarks(image, results)
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        if len(sequence) == 30:
          res = model.predict(np.expand_dims(sequence, axis=0))[0]
          name = actions[np.argmax(res)]
          cal = res[np.argmax(res)] * 100
          if cal <= 50:
            name = "Do not know"
          cv2.putText(image, name, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,0), 2)
          cv2.putText(image, str('%.2f' %(cal)) + " %", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
          cv2.imshow("{0}".format(name_video), image)
          vdo_writer.write(image)
        else:
          vdo_writer.write(frame)
          cv2.imshow("{0}".format(name_video), frame)
      if (cv2.waitKey(1) & 0xFF == ord('q')) or not ret:
        break
  vdo_writer.release()
  cap.release()
  cv2.destroyAllWindows()

In [29]:
play_video('WIN_5', model, actions)

1/1 [==============================] - 0s 48ms/step


In [10]:
play_video('Hungry', model, actions)

1/1 [==============================] - 0s 32ms/step


In [11]:
play_video('Sick', model, actions)

1/1 [==============================] - 0s 32ms/step


In [12]:
play_video('Sorry', model, actions)

1/1 [==============================] - 0s 38ms/step


In [13]:
play_video('Thank you', model, actions)

1/1 [==============================] - 0s 33ms/step


In [14]:
play_video('What', model, actions)

1/1 [==============================] - 0s 34ms/step


In [15]:
play_video('When', model, actions)

1/1 [==============================] - 0s 69ms/step


In [16]:
play_video('Where', model, actions)

1/1 [==============================] - 0s 33ms/step


In [17]:
play_video('Who', model, actions)

1/1 [==============================] - 0s 37ms/step


In [18]:
play_video('Why', model, actions)

1/1 [==============================] - 0s 36ms/step
